In [1]:
%load_ext autoreload
%autoreload 2
import jieba
import jieba.posseg as pseg
import nltk
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import torch
import re
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
import utils.LS_funcs as LS_funcs
ner_pipeline = pipeline(Tasks.named_entity_recognition, 'damo/nlp_raner_named-entity-recognition_chinese-base-news')


2025-02-14 15:32:54,933 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


2025-02-14 15:32:57,313 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-02-14 15:32:57,729 - modelscope - INFO - initiate model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-14 15:32:57,729 - modelscope - INFO - initiate model from location C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news.
2025-02-14 15:32:57,745 - modelscope - INFO - initialize model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-14 15:33:00,505 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\modelscope\utils\checkpoint.py:550: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will

2025-02-14 15:33:05,668 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-02-14 15:33:06,084 - modelscope - INFO - initiate model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-14 15:33:06,084 - modelscope - INFO - initiate model from location C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news.
2025-02-14 15:33:06,099 - modelscope - INFO - initialize model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-14 15:33:08,618 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\modelscope\utils\checkpoint.py:550: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will

In [29]:
with open("../data/BLCU/top_similar.pickle", 'rb') as handle:
    top_similar = pickle.load(handle)
with open("../data/BLCU/similarity_dict.pickle", 'rb') as handle:
    similarity_dict = pickle.load(handle)
with open("../data/HSK/HSK_levels.pickle", 'rb') as handle:
    hsk_dict = pickle.load(handle)

blcu = pd.read_csv('../data/BLCU/literature_wordfreq.release_UTF-8.txt', header = None, sep="\t",)
blcu.rename(columns={0:"character", 1:"frequency"}, inplace=True)
blcu.set_index("character", inplace=True)
blcu["frequency"] = blcu["frequency"].rank(pct=True)


In [ ]:
from evaluate import load
sari = load("sari")
sources=["About 95 species are currently accepted."]
predictions=["About 95 you now get in."]
references=[["About 95 species are now accepted."]]
sari.compute(sources=sources, predictions=predictions, references=references)

In [28]:
blcu.loc[blcu["character"]=='精辟']["frequency"].values[0]

np.float64(0.9041140338118308)

In [19]:
tokens = jieba.lcut(sentence)
tokens = [token for token in tokens if not re.match(r'^\W+$', token)] # remove punctuation

### NER on each token:
ner_output = ner_pipeline(sentence)['output'] # run NER pipeline, generate tokens
tokens_ner = list(set([d['span'] for d in ner_output if len(d['span'])>1])) # only collect NER longer than 1 character
tokens_no_ner = list(set(tokens) - set(tokens_ner)) # get all tokens that are NOT named entities


c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\modeling_utils.py:1113: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [4]:
sentence = "他的见解十分独到，对问题的分析非常精辟。"
LS_funcs.LS_pipeline(sentence, verbose=True)

2025-02-14 15:57:48,976 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


2025-02-14 15:57:51,195 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-02-14 15:57:51,611 - modelscope - INFO - initiate model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-14 15:57:51,611 - modelscope - INFO - initiate model from location C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news.
2025-02-14 15:57:51,624 - modelscope - INFO - initialize model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-14 15:57:54,244 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\modelscope\utils\checkpoint.py:550: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will

Best index is 3
Original sentence:
 他的见解十分独到，对问题的分析非常精辟。
Simplified sentence:
 他的看法十分独特，对问题的分析非常精彩。
Complex word:  精辟
Candidates:  ['精彩', '精妙', '深刻', '独到', '绝妙', '透彻', '巧妙', '精悍', '详尽', '精深']
Complex word:  见解
Candidates:  ['识见', '见闻', '见识', '看法']
Complex word:  独到
Candidates:  ['独特', '独家', '特有', '别致', '奇特', '特殊', '特异']


'他的看法十分独特，对问题的分析非常精彩。'

In [11]:
list(pseg.cut("精辟"))[0]

pair('精辟', 'v')

In [10]:
list(pseg.cut("清楚"))[0].flag

'a'

In [8]:
similarity_dict["精辟"]

['独到', '透彻']